In [1]:
import os 
import flask
import pandas as pd
from engine import TextClassificationEngine, TopicModellingModel
from utilities import send_request_reddit_get_new_post
from pyspark.sql import SparkSession
from pyspark.sql.functions import max, col, udf

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
spark = SparkSession.builder \
           .appName('Web Appp') \
           .config("spark.jars", "mysql-connector-j-8.0.32.jar")\
           .config("spark.driver.memory", "6g") \
           .config("spark.executor.memory", "8g") \
           .getOrCreate()

In [3]:
path = os.getcwd()

In [4]:
label_engine = TextClassificationEngine(spark)

INFO:engine:Starting up text classification engine: 
INFO:engine:Loading labled data...
INFO:engine:Loading completed
INFO:engine:Preprocessing data...
INFO:engine:Preprocessing completed
INFO:engine:Vectorize data...
INFO:engine:Vectorization completed
INFO:engine:Training text classification model...
INFO:engine:Text classification model built!


In [5]:
username = 'AutomaticPepper1040'
url = f'https://oauth.reddit.com/user/{username}/submitted'
response, status = send_request_reddit_get_new_post(url)
total_post = pd.DataFrame(columns = ['post_id', 'descriptions'])
for post in response['data']['children']:
    post_id = post['data']['id']
    descriptions = post['data']['title']
    total_post_aux = pd.DataFrame({'post_id': [post_id], 'descriptions': [descriptions]})
    total_post = pd.concat([total_post_aux, total_post], ignore_index = True, axis = 0)

predicted_label = label_engine.predict_label(total_post)
grouped_label = predicted_label.groupBy("label_name").count()
max_count = grouped_label.agg(max("count")).first()[0]

highest_count_groups = grouped_label.filter(col('count') == max_count)
label_name = highest_count_groups.select('label_name').first()[0]
considered_post = predicted_label.filter(col('label_name') == label_name)
topic_engine = TopicModellingModel(spark, label_name)

predicted_topic = topic_engine.predict_topic(considered_post)

grouped_topic = predicted_topic.groupBy("topic").count()
max_count = grouped_topic.agg(max("count")).first()[0]

highest_count_groups = grouped_topic.filter(col('count') == max_count)
topic = highest_count_groups.select('topic').first()[0]
results = topic_engine.get_recommendation(topic)
recommendations = []
for result in results: 
    recommendation = []
    descriptions = result[2]
    created_utc = result[3]
    source_url = result[4]
    post_url = result[5]
    category = result[6]
    recommendation.append(descriptions)
    recommendation.append(created_utc)
    recommendation.append(source_url)
    recommendation.append(post_url)
    recommendation.append(category)
    recommendations.append(recommendation)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
INFO:engine:Starting up model LDA Business: 
INFO:engine:Loading data...
INFO:engine:Loading completed
INFO:engine:Preprocessing data...
INFO:engine:Preprocessing completed
INFO:engine:LDA Business model built!


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/jovyan/work/web_app/engine.py", line 159, in <lambda>
    max_index = udf(lambda x: x.index(max(x)) if x is not None else None, IntegerType())
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 195, in max
    return _invoke_function_over_columns("max", col)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 93, in _invoke_function_over_columns
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 93, in <genexpr>
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/sql/column.py", line 65, in _to_java_column
    raise TypeError(
TypeError: Invalid argument, not a string or column: [0.02001713277407433, 0

In [ ]:
# app = flask.Flask(__name__, template_folder = os.path.join(path, 'template'))
# @app.route('/', methods = ['GET', 'POST'])
# def main():
#     if flask.request.method == 'GET':
#         return flask.render_template('index.html')
            
#     if flask.request.method == 'POST':
#         username = flask.request.form['user_id']
#         url = f'https://oauth.reddit.com/user/{username}/submitted'
#         response, status = send_request_reddit_get_new_post(url)
#         total_post = pd.DataFrame(columns = ['post_id', 'descriptions'])
#         for post in response['data']['children']:
#             post_id = post['data']['id']
#             descriptions = post['data']['title']
#             total_post_aux = pd.DataFrame({'post_id': [post_id], 'descriptions': [descriptions]})
#             total_post = pd.concat([total_post_aux, total_post], ignore_index = True, axis = 0)
            
#         predicted_label = label_engine.predict_label(total_post)
#         grouped_label = predicted_label.groupBy("label_name").count()
#         max_count = grouped_label.agg(max("count")).first()[0]

#         highest_count_groups = grouped_label.filter(col('count') == max_count)
#         label_name = highest_count_groups.select('label_name').first()[0]
#         considered_post = predicted_label.filter(col('label_name') == label_name)
#         topic_engine = TopicModellingModel(spark, label_name)
        
#         predicted_topic = topic_engine.predict_topic(considered_post)
        
#         grouped_topic = predicted_topic.groupBy("topic").count()
#         max_count = grouped_topic.agg(max("count")).first()[0]

#         highest_count_groups = grouped_label.filter(col('count') == max_count)
#         topic = highest_count_groups.select('topic').first()[0]
#         results = topic_engine.get_recommendation(topic)
#         recommendations = []
#         for result in results: 
#             recommendation = []
#             descriptions = result[2]
#             created_utc = result[3]
#             source_url = result[4]
#             post_url = result[5]
#             category = result[6]
#             recommendation.append(descriptions)
#             recommendation.append(created_utc)
#             recommendation.append(source_url)
#             recommendation.append(post_url)
#             recommendation.append(category)
#             recommendations.append(recommendation)
#         return flask.render_template('result.html', recommendations = recommendations)

# if __name__ == '__main__':
#     app.run(host="0.0.0.0")